Adding "is_artifact" column to Wuhan (8989)

cf. here: https://github.com/martinfleis/urban-block-artifacts/blob/main/code/08_validation.ipynb 

In [1]:
import json

import dask_geopandas
import geopandas as gpd

In [3]:
with open("../data/04_peaks_results.json") as f:
    results = json.load(f)

myid = 8989
city = "Wuhan"
polygons = gpd.read_parquet(f"../data/{myid}/polygons/")

# find predicted artifacts
option = "circular_compactness_index"
threshold = results[city][option]["threshold"]
polygons["is_artifact"] = False  # set default to False
polygons.loc[polygons["circular_compactness_index"] <= threshold, "is_artifact"] = (
    True  # set to True for polygons where index is below the threshold
)

In [4]:
# save polygons to a partitioned GeoParquet, overwriting original polygons
polygons = dask_geopandas.from_geopandas(polygons, npartitions=10)
polygons.to_parquet(f"../data/{int(myid)}/polygons/")